In [ ]:
## 广度搜索-与“说”相似的词
from gensim.models import Word2Vec, KeyedVectors
from collections import defaultdict

## Get similar words to said - dic: 'word': scanned_times
def similar_said_words(initial_words, model):
    un_scanned_words = initial_words
    scanned_words = defaultdict(int)
    max_size = 400
    
    while un_scanned_words and len(scanned_words)<max_size:
        node = un_scanned_words.pop(0)
        new_added_words  = [w for w, _ in model.wv.most_similar(node, topn=15)]
        un_scanned_words += new_added_words
        scanned_words[node] += 1
    return scanned_words

def get_similar_words():
    words_list = ['说', '强调', '表示', '告诉'] 
    model = KeyedVectors.load_word2vec_format(r'D:\Python\NLP\news_word2vce.bin', binary=True)
    #model = Word2Vec.load(model_path)
    related_words = similar_said_words(words_list, model)
    related_words = sorted(related_words.items(), key=lambda x: x[1], reverse=True)
    related_said_words = [i[0] for i in related_words]
    return related_said_words
    

##在搜索结果的基础上，加入网上搜集到的与说相关的词 
other_speak_words =  ['表示', '说','回复', '指出', '认为', '坦言', '告诉', '强调', '称', '直言', '普遍认为', '介绍', '透露', '重申', '呼吁', '说道', '感叹', '地说', '写道',
                      '中称', '证实', '还称', '猜测', '暗示', '感慨', '热议', '敦促', '指责', '声称', '主张', '反对', '批评', '表态', '中说', '承认', '却说', '感触',
                      '提到', '所说', '引述', '质疑', '抨击','回应', '分析']
##将相似单词保存到txt文件    
def words_save():
    #model_path = r'D:\Python\NLP\word2vec_779845.bin'
    saved_file_path = r'D:\Python\NLP\said_similar_words.txt'
    said_similar_words = get_similar_words()
    said_similar_words += other_speak_words 
    string = ' '.join(set(said_similar_words))
    with open(saved_file_path, 'w', encoding='utf-8') as f:
        f.write(string)
        print('Saved !')
    return True

##读取相似单词
def load_words_list():
    file_path = r'D:\Python\NLP\said_similar_words.txt'
    with open(file_path, 'r', encoding='utf-8') as f:
        string = f.readlines()
        string = string[0].split(' ')
        return string
    
words_save()
          


In [1]:
##加载词向量模型
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format(r'D:\Python\NLP\news_word2vce.bin', binary=True)

In [17]:
import numpy as np
from numpy import linalg
import re
import jieba
from pyltp import Postagger, Parser, NamedEntityRecognizer, Segmentor, SentenceSplitter
#import functools 
from collections import defaultdict
#import logging
import pandas as pd


class speech_extract():
    
    def __init__(self):
        
        self.model = model
        self.words_dict = self.model.wv.vocab
        self.embedding_dims = self.model.vector_size
        
        ##postag, parse, ner, cws 模型路径
        self.pos_model_path = r'D:\Python\NLP\ltp_models\pos.model' 
        self.par_model_path = r'D:\Python\NLP\ltp_models\parser.model'
        self.ner_model_path = r'D:\Python\NLP\ltp_models\ner.model'
        self.cws_model_path = r'D:\Python\NLP\ltp_models\cws.model'
        self.pisrl_model_path = r'D:\Python\NLP\ltp_models\pisrl.model'
    
    ## jieba分词
    def cut_words(self, sentences):
        
        words_list = []
        words_list = jieba.lcut(sentences)
        
        return words_list
    
    ##Pyltp segment分词，在试验的大部分news中，segment分词效果比jieba好
    def segment_words(self, sentences):
        
        segmentor = Segmentor()
        segmentor.load(self.cws_model_path)
        word_list = list(segmentor.segment(sentences))
        segmentor.release()
        
        return word_list
    
    ##段落分句
    def sentence_cut(self, sentences: str):
        
        sentences_bag = SentenceSplitter.split(sentences)
        sentences = [sen for sen in sentences_bag if len(sen)>0]
        
        return sentences
    
    
    ##词性标注
    def get_postags(self, words_list):
        
        postagger = Postagger()
        postagger.load(self.pos_model_path)
        postagger_list = list(postagger.postag(words_list))
        postagger.release()
        
        return postagger_list
    
    ##句法依存关系提取
    def get_parse_list(self, words_list, postagger_list):
        
        parser = Parser()
        parser.load(self.par_model_path)
        arcs = parser.parse(words_list, postagger_list)
        arc_list = [(arc.head, arc.relation) for arc in arcs]
        parser.release()
        
        return arc_list
    
    ##命名实体识别
    def get_ner(self, words_list, postagger_list):
        
        recognizer = NamedEntityRecognizer()
        recognizer.load(self.ner_model_path)
        ner_tags = recognizer.recognize(words_list, postagger_list)
        recognizer.release()
        
        return ner_tags
    
    ##获取某段news句法依存关系
    def get_paragraph_parse(self, para):
        
        word_list = self.cut_words(para)
        para_postagger_list = self.get_postags(word_list)
        get_para_parse_list = self.get_parse_list(word_list, para_postagger_list)
        para_parse_list = [(word_list[i], get_para_parse_list[i]) for i in range(len(word_list))]
        
        return para_parse_list
    
    ##调取和“说”相似的词txt文件
    def get_similar_words(self):
        
        with open(r'D:\Python\NLP\said_similar_words.txt', 'r', encoding='utf-8') as f:
            string = f.readlines()
            string = string[0].split(' ')
            return string
    
    ##SIF方法定义句向量
    def SIF_sentence_vector(self, sentence, a: float=1e-3, no_appear_word_freq=0.0001):
     
        ##words词总数量
        vocab_counts = 0
        for word in self.words_dict:
            vocab_counts += self.words_dict[word].count
            
        pattern = re.compile("[^\u4e00-\u9fa5^a-z^A-Z]")
        sentence = pattern.sub('', sentence)
        sentence_words_list = self.cut_words(sentence)
        sentence_len = len(sentence_words_list)
        sen_vector = np.zeros(self.embedding_dims)
        
        for word in sentence_words_list:
            
            if word in self.words_dict:
                a_refresh = a / (a + self.words_dict[word].count / vocab_counts)
                sen_vector = np.add(sen_vector, np.multiply(a_refresh, self.model.wv[word]))
                
            if word not in self.words_dict:
                a_refresh = a / (a + no_appear_word_freq)  
                sen_vector = np.add(sen_vector, np.multiply(a_refresh, np.zeros(self.embedding_dims)))
                
        sentence_vector = np.divide(sen_vector, sentence_len)
        
        return sentence_vector
    
    ## 余弦距离
    def cosine_dis(self,vect_1, vect_2):
        
        vect_1 = np.mat(vect_1)
        vect_2 = np.mat(vect_2)
        neiji = float(vect_1 * vect_2.T)
        denorm = linalg.norm(vect_1) + linalg.norm(vect_2)
        cos_distance = 0.5 + 0.5 * denorm / neiji
        
        return cos_distance
    
    ##计算两个句子的相似度
    def sentence_distance(self, sentence1, sentence2):
        
        vector_1 = self.SIF_sentence_vector(sentence1, 1e-3, 0.0001)
        vector_2 = self.SIF_sentence_vector(sentence2, 1e-3, 0.0001)
        
        return self.cosine_dis(vector_1, vector_2)
    
    ##提取出一段news中，所有和“说”相关的词语
    def say_words_index(self, parse_list):
        
        saywords_index = []
        say_similar_word = self.get_similar_words()
        
        for item in parse_list:
            if item[1][1] == 'SBV':
                verb_word_index = item[1][0] - 1
                if parse_list[verb_word_index][0] in say_similar_word:
                    saywords_index.append(verb_word_index)
                    
        return sorted(set(saywords_index))
    
    def common_elements(self, list1, list2):
        
        common_list = [a for a in list1 if a in list2]
        
        if common_list:
            return True
        else:
            return False
    
    ## 获取“说”的句子
    def get_say_sentence(self, parse_list, say_index):
        
        speech_start = 0
        speech_end = len(parse_list) - 1
        
        ## 前面话语的结束位置
        for i in range(say_index, -1, -1):
            if parse_list[i][0] in ['“', '”', '，', '。', '！', '？']:
                speech_start = i + 1
                break
        
        ##后面话语的开始位置
        for j in range(say_index, len(parse_list)+1):
            if parse_list[j][1][1] == 'WP' or parse_list[j][0] == '：':
                speech_end = j + 1
                break
                
        speech_words = [parse_list[k][0] for k in range(speech_start, speech_end)]
        #print(speech_words)
        speech_sentence = ''.join(speech_words)
        
        content_before = ''
        content_before_end = speech_start - 1
        #sentence_end_index = 0
        #punc_marks = [',', '。', '！', '？']
        
        if content_before_end > 0:
            if parse_list[content_before_end][0] == '”':
            ##判断前面话语”前是否有中断，如有，则content_before=''
                for index in range(content_before_end, -1, -1):
                    if parse_list[index][0] == '“':    ##以“”来判断句子的起始结束，需进一步考虑是否涵盖其他情况
                        content_before = ''.join([parse_list[key][0] for key in range(index, content_before_end+1)])
                        #print(content_before)
                        break
              
        content_after = ''                
        content_after_start = speech_end
        
        ##通过“”或者, 。来判断句子的起始结束
        if content_after_start < len(parse_list) and parse_list[content_after_start-1][0] != '。':
            if parse_list[content_after_start][0] == '“' or '，':
                end_mark = '”'
            else:
                end_mark = '。'
            for key_ in range(content_after_start, len(parse_list)):
                if parse_list[key_][0] == end_mark:
                    content_after = ''.join([parse_list[key_1][0] for key_1 in range(content_after_start, key_+1)])
                    #print(content_after)
                    break
                #end_punc = '”'
            
            ##通过计算前后句子的余弦相似度来判断是否为话语延续
            else:
                after_words = ''.join([parse_list[key_2][0] for key_2 in range(content_after_start, len(parse_list))])
                sentences = self.sentence_cut(after_words)
                first_sentence = sentences.pop(0)
                content_after_list = [first_sentence]
                while sentences:
                    next_sentence = sentences.pop(0)
                    if self.sentence_distance(first_sentence, next_sentence)>0.8:
                        content_after_list.append(next_sentence)
                    else:
                        break
                content_after = ''.join(content_after_list)
                #print(content_after)
 
        return speech_sentence, speech_words, content_before, content_after

    ##获取说话的人和言语
    def get_speech(self, news):
        
        news = re.sub('\\\\n|[\n\u3000\r]', '', news)
        news_words_parse = self.get_paragraph_parse(news)
        speak_word_indexes = self.say_words_index(news_words_parse)
        extract_results = []
        
        for word_index in speak_word_indexes:
            speak_content_dic = {}
            speech_sentence, speech_words, content_before, content_after = self.get_say_sentence(news_words_parse, word_index)
            #print(speech_words)
            postag_list = self.get_postags(speech_words)
            ner_list = self.get_ner(speech_words, postag_list)
            #ner_list = [(index, ner_label)]
            #print(list(ner_list))
            
            ner_label_list = [(index, ner_label) for index, ner_label in enumerate(ner_list)]
            speak_name_index = [item[0] for item in ner_label_list if item[1]=='S-Nh']   ##如果为单独人名，则返回该人名，未考虑“他说，她说”第三人称情况，需优化
            if not speak_name_index: continue
            speak_names_list = speech_words[speak_name_index[0]:speak_name_index[-1]+1] ##考虑好几个人同时说
            speak_names = ''.join(speak_names_list)
            
            for id in range(speak_name_index[0]-1, -1, -1):
                if news_words_parse[id][0] not in ['。', '”', '“', '！', '？']:  ##考虑提取人名前的修饰词和状语等，例如XXX大学教授，XXX学会会长，方法有待完善（通过NER来判断）
                    speak_names = speech_words[id] + speak_names
                else:
                    break
            
                
                #if ner_label_list[id][1] not in ['B-Ni', 'E-Ni', '、' 'B-Ni', 'I-Ni', 'E-Ni']:
                    #break
            
            speak_content = ''
            if content_before:
                speak_content += content_before
            if content_after:
                speak_content += ' ' + content_after
                
            speak_content_dic[speak_names] = speak_content  合并人名和言论到字典
                
            extract_results.append(speak_content_dic)
            
        return extract_results

In [18]:
test_news = """
2020年，是决胜全面建成小康社会关键之年，也是向脱贫攻坚发起总攻、夺取全胜的决战之年。疫情防控是特殊考验，脱贫攻坚是时代使命。打好打赢这两场大战，事关如期全面建成小康社会全局。
这是一场重要时期围绕重要目标召开的重要会议。
新冠肺炎疫情当前，脱贫攻坚能不能缓一缓、等一等？在前不久中央刚刚举行的统筹推进新冠肺炎疫情防控和经济社会发展工作部署会上，习近平总书记讲话掷地有声：“坚决完成脱贫攻坚任务。”
在这次会议上，习近平总书记再度强调了“坚决”二字。坚决克服新冠肺炎疫情影响，坚决夺取脱贫攻坚战全面胜利，坚决完成这项对中华民族、对人类都具有重大意义的伟业。
“习近平总书记对加强党的领导、高质量完成脱贫攻坚目标任务提出明确要求，为决战决胜脱贫攻坚注入强大信心和力量。”全国政协常委、中国税务学会副会长张连起对此表示。
关于决战决胜脱贫攻坚座谈会，习近平总书记坦言，自己的计划是，从年初就考虑到外地考察，把有关地方特别是还没有摘帽的贫困县所有负责同志都请到一起开个会，研究决战脱贫攻坚工作部署。如今受新冠肺炎疫情影响，这场会迟迟没开。他真诚地说：“也考虑过等疫情得到有效控制后再到地方去开，但又觉得今年满打满算还有不到10个月的时间，按日子算就是300天……”
逆水行舟用力撑，一篙松劲退千寻。“这是一场硬仗，越到最后越要紧绷这根弦，不能停顿、不能大意、不能放松。”习近平总书记强调，“各级党委（党组）一定要履职尽责、不辱使命”。
“总书记讲话的核心意思是战‘疫’不放松，战贫要抓紧，这是面向全国人民的承诺，脱贫攻坚一定要高质量完成。”张连起认为，“一方面要做好分区分级精准防控，尽快实现经济社会发展的正常生活、正常秩序，另一方面脱贫攻坚一点也不能放松。”
党的十八大以来，在以习近平同志为核心的党中央坚强领导下，在全党全国全社会共同努力下，我国脱贫攻坚取得决定性成就：贫困人口从2012年年底的9899万人减到2019年年底的551万人，贫困发生率由10.2%降至0.6%，区域性整体贫困基本得到解决，脱贫攻坚目标任务胜利在望。
在3月6日的会议上，习近平总书记指出，“目前看，脱贫进度符合预期，成就举世瞩目。”
脱贫攻坚目标任务接近完成、贫困群众收入水平大幅度提高、贫困地区基本生产生活条件明显改善、贫困地区经济社会发展明显加快、贫困治理能力明显提升、中国减贫方案和减贫成就得到国际社会普遍认可……每一项成绩的取得，都凝聚了全党全国各族人民智慧和心血，是广大干部群众扎扎实实干出来的。
“全面建成小康社会，打赢脱贫攻坚战是我们党向人民、向历史作出的庄严承诺，是我们党的历史责任。为了完成这一艰巨的历史任务，确保如期实现全面建成小康社会的目标，各级政府层层签了完成任务的军令状，不获全胜不能收兵。”中央党校（国家行政学院）督学组督学、教授、博导洪向华表示,“总书记的讲话指明方向，振奋人心，让我们有信心有决心坚决战胜疫情，顺利如期完成全面建成小康社会的目标。”
    """

news_extract = speech_extract()
extract_results = news_extract.get_speech(test_news)
print(extract_results)
#df = pd.DataFrame.from_dict(extract_results, orient='index', columns=['言论'])
#df = df.reset_index().rename(columns={'index': '人物'})
#df.head()
#print(df)


C:\Users\liang\.conda\envs\py36\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


[{'习近平': ' “坚决完成脱贫攻坚任务。”'}, {'习近平': ' 坚决”'}, {'习近平': ' 为决战决胜脱贫攻坚注入强大信心和力量。”'}, {'全国政协常委、中国税务学会副会长张连起': '“习近平总书记对加强党的领导、高质量完成脱贫攻坚目标任务提出明确要求，为决战决胜脱贫攻坚注入强大信心和力量。”'}, {'习近平': '“这是一场硬仗，越到最后越要紧绷这根弦，不能停顿、不能大意、不能放松。” “各级党委（党组）一定要履职尽责、不辱使命”'}, {'习近平': ' “目前看，脱贫进度符合预期，成就举世瞩目。”'}, {'中央党校（国家行政学院）督学组督学、教授、博导洪向华': '“全面建成小康社会，打赢脱贫攻坚战是我们党向人民、向历史作出的庄严承诺，是我们党的历史责任。为了完成这一艰巨的历史任务，确保如期实现全面建成小康社会的目标，各级政府层层签了完成任务的军令状，不获全胜不能收兵。” “总书记的讲话指明方向，振奋人心，让我们有信心有决心坚决战胜疫情，顺利如期完成全面建成小康社会的目标。”'}]


In [109]:
news_extract_results = defaultdict(list)

[news_extract_results[k].append(v) for i in extract_results for k, v in i.items()]

news_extract_results_final = dict(news_extract_results))

df = pd.DataFrame.from_dict(news_extract_results_final, orient='index', columns=['言论'])
df = df.reset_index().rename(columns={'index': '人物'})
df.head()
#print()


{'习近平': [' “坚决完成脱贫攻坚任务。”', ' 坚决”', ' 为决战决胜脱贫攻坚注入强大信心和力量。”', '“这是一场硬仗，越到最后越要紧绷这根弦，不能停顿、不能大意、不能放松。” “各级党委（党组）一定要履职尽责、不辱使命”', ' “目前看，脱贫进度符合预期，成就举世瞩目。”'], '全国政协常委、中国税务学会副会长张连起': ['“习近平总书记对加强党的领导、高质量完成脱贫攻坚目标任务提出明确要求，为决战决胜脱贫攻坚注入强大信心和力量。”'], '中央党校（国家行政学院）督学组督学、教授、博导洪向华': ['“全面建成小康社会，打赢脱贫攻坚战是我们党向人民、向历史作出的庄严承诺，是我们党的历史责任。为了完成这一艰巨的历史任务，确保如期实现全面建成小康社会的目标，各级政府层层签了完成任务的军令状，不获全胜不能收兵。” “总书记的讲话指明方向，振奋人心，让我们有信心有决心坚决战胜疫情，顺利如期完成全面建成小康社会的目标。”']}
